In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import scipy.signal as signal
import scipy.integrate as integrate
import matplotlib.pyplot as plt
from BinaryFileUnpack import BinaryFileUnpack
import os
import warnings

In [11]:
def get_omega(start:float, end:float, Data:np.ndarray, obj:BinaryFileUnpack) -> tuple:
    fs = obj.fs
    s = int(start * fs)
    e = int(end * fs)
    t = obj.time[s:e]

    # Use regression to find angular frequency
    # If regression fails, then use spectra
    try:
        P = Data
        # generate good initial guesses of the parameters
        peaks = signal.find_peaks(P,prominence=.0005)
        # peaks[0] will be an array of the indices corresponding to the peak location
        # plt.figure()
        # plt.plot(t,P)
        # plt.scatter(t[peaks[0]],P[peaks[0]])
        # guess the period as the average separation between peaks
        period_guess = np.mean( np.diff(t[peaks[0]]) )
        lag_guess = t[peaks[0][0]] # guess that the first maximum occurs at the time of the first peak

        def line(t,a,b):
            return a*t+b

        linear_fit = curve_fit(line,t,P)
        a,b = linear_fit[0]
        # plt.plot(t,a*t+b,'r--')
        # guess the amplitude of the initial perturbation:
        amplitude_guess = P[ peaks[0][0] ] - a*t[peaks[0][0]] - b

        def damped_oscillator(time,amplitude,period,lag,decaytime,slope,offset): 
            return amplitude * np.cos(2.0*np.pi*((time-lag)/period)) * np.exp(-(time-lag)/decaytime) + slope*(time) + offset

        initial_guess = [amplitude_guess,period_guess,lag_guess,(t[-1]-t[0])/2.0,a,b]
        popt, pcov = curve_fit(damped_oscillator,t,P,p0=initial_guess)

        omega = 2*np.pi/popt[1]
        sigma_omega = np.sqrt(pcov[1][1])/popt[1] * omega
    except:
        # Don't use spectra method, use Lomb-Scargle method for deterministic frequencies
        omegas = np.linspace(2, 9, 2000)
        pgram = signal.lombscargle(t, Data, omegas, precenter=True)
        power = 10*np.log10(pgram)
        
        max_ind = np.argmax(power)
        omega = omegas[max_ind]

        # Std. dev. (sigma) for omega
        # Use normal distribution with deciBel conversion with form
        # N(f) = N(fmax) - 10/ln(10) * ((f-fmax)/sigma)^2
        # A point is chosen 10 indices from fmax, such that
        # sigma = (10*res) * sqrt( (10/ln(10)) / (N(fmax) - N(fmax - 10*res)) )
        # res is the gap in frequency data
        res = omegas[1]-omegas[0]
        delta = 10*res
        sigma_omega = delta * np.sqrt( (10/np.log(10)) / (power[max_ind] - power[max_ind-10]) )
    return omega, sigma_omega

# (Not in use) Specify base pressures and H based on conduit diameter and sensor index
def spec_params(cond_diam, sens_ind, path, x):
    if sens_ind == 2:
        if cond_diam == 1:
            pres = 1.2589916544823685 # bar
            pres_std = 5.423258097482985e-05 # bar
            H = 58 # cm
        elif cond_diam == 2:
            pres = 1.3079672852169744 # bar
            pres_std = 0.00014948905833631553 # bar
            H = 46 # cm
        # Correction due to atmospheric calibration
        pres = pres*0.874466484 - 0.0786998490
        pres_std *= 0.874466484
    elif sens_ind == 0:
        # Use extracted values for sensor 1 calibration
        df_cal = pd.read_csv(f"{path}/../../sensor1_calibration.csv")
        pres = df_cal.iloc[0][0] + x*df_cal.iloc[1][0]
        pres_std = np.sqrt(df_cal.iloc[0][1]**2 + x*df_cal.iloc[1][1]**2)
        
        if cond_diam == 1:
            H = 58 # cm
        elif cond_diam == 2:
            H = 46
        # Correction due to atmospheric calibration
        pres = pres*1.05029088 - 0.298257335
        pres_std *= 1.05029088
    
    return pres, pres_std, H

df = pd.read_csv("calibration_data_03-31.csv")
def getY(pressure, sens_ind, H=None):
    # Linear model
    f = lambda x, b0, b1: b0 + b1*x
    
    if sens_ind == 0:
        # assert H is not None, "H needs to be specified"
        ytest = df["Y* (cm)"].to_numpy()
        P1 = df["P1 (Raw)"].to_numpy()
        p1opt, p1cov = curve_fit(f, P1, ytest, absolute_sigma=True)
        y = f(pressure, *p1opt)
        y_stdev = 0
        for i in range(len(ytest)):
            y_stdev += (ytest[i] - f(P1[i], *p1opt))**2 / len(ytest)
    # (Only one in use)
    elif sens_ind == 2:
        ytest = df["Y* (cm)"].to_numpy()
        P3 = df["P3 (Raw)"].to_numpy()
        p3opt, p3cov = curve_fit(f, P3, ytest, absolute_sigma=True)
        y = f(pressure, *p3opt)
        y_stdev = 0
        for i in range(len(ytest)):
            y_stdev += (ytest[i] - f(P3[i], *p3opt))**2 / len(ytest)


    return y, y_stdev
    


In [33]:
# Short 1in conduit
path = r"C:\Users\akyap\OneDrive\Documents\Academics\Research\LDEO Geysers\files\02-17"
X65 = [
    (9.1, 18, 3),
    (38.5, 60, 3),
    (80.1, 95, 3),
    (149.5, 160, 3),
    (162.1, 175, 0),
    (224.2, 239, 0),
    (265.4, 280, 0),
    (325, 340, 0),
    (435, 445, 0),
    (505, 515, 0),
    (600, 610, 0),
    (689, 699, 2),
    (766, 776, 2),
    (805, 814, 2),
    (872, 882, 2),
    (930.5, 940, 2),
    (964, 972.5, 2),
    (1007.5, 1016, 2),
]

tests_1inch_2 = {
    'cond_diam,H' : (1,29),
    'X65cm' : (X65, path+r"\HotWaterOsc_X65cm-20230217-19-19-42.bin"),
}


In [36]:
# Hot 2-inch second go
path = r"C:\Users\akyap\OneDrive\Documents\Academics\Research\LDEO Geysers\files\03-24"
X68 = [
    (50, 65, 0),
    (97.6, 111, 0),
    (198.3, 211, 0),
    (340.1, 355, 0),
    (421.7, 436, 0),
    (473.5, 485, 0),
    (502, 515, 0),
    (561.8, 576, 0),
    (615.2, 627, 0),
    (686, 698, 0),
    (744, 756, 0),
    (798.8, 808, 0),
    (970.2, 980, 0),
    (1056, 1065, 0),
]

tests_2inch_2 = {
    'cond_diam,H' : (2,46),
    'X68cm' : (X68, path+r"\HotWater_X68-20230324-18-05-22.bin"),
}

The following data is for 1-inch diameter conduit.

In [23]:
path = r"C:\Users\akyap\OneDrive\Documents\Academics\Research\LDEO Geysers\files\tests\hot-water-1in"
X38cm = [
    (92.6, 112.4, 2),
]

X51cm_0 = [
    (26.7, 35, 2),
    # Oscillation from ~ 51 to 55 sec unusable
]

X51cm_1 = [
    (11.5, 24.5, 2),
    (49.4, 59.8, 2),
    (71.8, 82.0, 2),
    (97, 109, 2),
    (137, 146.9, 2)
]

X51cm_2 = [
    (31.2, 45.2, 2),
    (56, 67.7, 2),
    (56, 67, 2),
    (77.6, 90.3, 2),
    (111.6, 129, 2),
    (139.5, 149.5, 2),
    (174.75, 186, 2)
]

X60cm = [
    (14.9, 28.0, 2),
    (53.7, 64.2, 2),
    (77.5, 90, 2),
    (133.7, 147.2, 2),
    (175, 188, 2),
    (233, 250.5, 2),
]

X66cm = [
    (24.6, 38.3, 2),
    (60.8, 72.0, 2),
    (93.15, 110.6, 2),
    (130, 140.2, 2),
    (158, 173, 2),
    (215, 228.2, 2),
    (261.5, 274, 2),
    (291, 302.2, 2),
    (322, 333, 2)
]

X71cm = [
    (24.7, 34.8, 2),
    (54, 64.3, 2),
    (72.8, 82.6, 2),
    (98.8, 108.5, 2),
    (115, 123, 2),
    (153.4, 163.0, 2),
    (189.5, 200, 2),
]

tests_1inch_1 = {
    'cond_diam,H' : (1,58),
    'X38cm':(X38cm, path+r'\X38cm\HotWaterOsc_X38cm_YNA-20220805-18-25-16.bin'),
    'X51cm_0':(X51cm_0, path+r'\X51cm\HotWaterOsc_X51cm_YNA-20220805-18-49-15.bin'),
    'X51cm_1':(X51cm_1, path+r'\X51cm\HotWaterOsc_X51cm_YNA-20220805-18-52-04.bin'),
    'X51cm_2':(X51cm_2, path+r'\X51cm\HotWaterOsc_X51cm_YNA-20220805-18-56-47.bin'),
    'X60cm':(X60cm, path+r'\X60cm\HotWaterOsc_X60cm_YNA-20220805-19-17-07.bin'),
    'X66cm':(X66cm, path+r'\X66cm\HotWaterOsc_X66cm_YNA-20220805-19-29-43.bin'),
    'X71cm':(X71cm, path+r'\X71cm\HotWaterOsc_X71cm_YNA-20220805-19-45-56.bin')
}

The following data is for 2-inch diameter conduit.

In [16]:
path = r"C:\Users\akyap\OneDrive\Documents\Academics\Research\LDEO Geysers\files\tests\hot-water-2in"
X49cm = [
    (39.6,   50.0, 2),
    (60.2,   70.0, 2),
    (80.1,   90.0, 2),
    (107.7, 116.0, 2),
    (137.8, 145.0, 0),
    (155.2, 163.2, 0),
    (309.8, 318.0, 2),
    (331.7, 340.0, 2),
    (351.5, 362.0, 2),
    (370.0, 380.0, 2),
    (397.7, 408.0, 0),
]

X64cm = [
    (54.4,   64.0, 0),
    (77.1,   83.0, 0),
    (109.0, 115.0, 0),
    (183.6, 189.5, 2),
    (213.0, 220.0, 0),
    (229.5, 235.6, 2),
    (241.6, 245.6, 0),
    (261.0, 270.2, 2),
]

tests_2inch_1 = {
    'cond_diam,H' : (2,46),
    'X49cm' : (X49cm, path+r'\HotWater_X49cm-20221104-20-34-41.bin'),
    'X64cm' : (X64cm, path+r'\HotWater_X64cm-20221104-20-23-34.bin')
}

In [37]:
# Defining global physical parameters
Hb = 76  # cm
th  = 8.1 # cm

# tests dictionary should be in format of 
# tests = { 'X<x-value>':(<data_arr>, filepath), ... }
cond_diam = 0
tests = tests_2inch_2

test_summary = ['X (cm),Y (cm), start(s), end(s), omega, stdev-Y (cm), stdev-omega, sensor used'] 

for i, test in enumerate(tests):
    if i == 0:
        cond_diam, H = tests[test]
    else:
        filepath = tests[test][1]
        x = int(test[1:3])
        obj = BinaryFileUnpack(filepath)
        arr = tests[test][0]
        for i in range(len(arr)):
            # Get parameters for oscillation from dictionary
            start_time = arr[i][0]
            end_time = arr[i][1]
            sens_ind = arr[i][2]
            t = end_time - start_time
            # Index position for start and end times
            s = int(start_time * obj.fs)
            e = int(end_time * obj.fs)
            
            # Filter the high-frequency noise from data
            P = obj.P[sens_ind, s:e]
            bound_freq = 2.2
            nyq_freq = obj.fs // 2
            b, a = signal.butter(1, bound_freq/nyq_freq, 'lowpass')
            filteredP = signal.filtfilt(b, a, P)
            # Get the period
            omega, omega_stdev = get_omega(start_time, end_time, filteredP, obj)
            
            # Determine base pressures with provided sensor index
            # Determine H with provided conduit diameter

            # -- Make other fun here -- #
            # Obtain average h_y values for experiment
            if sens_ind != 2:
                P = obj.P[2, s:e]
                bound_freq = 2.2
                nyq_freq = obj.fs // 2
                b, a = signal.butter(1, bound_freq/nyq_freq, 'lowpass')
                filteredP = signal.filtfilt(b, a, P)
            y, y_stddev = getY(np.mean(filteredP), 2, H)
            # ------------------------- #

            # Convert to X and Y as defined in Max's illustration
            # X = x - (Hb - H)
            # Y = y + H, th is already included in y.
            test_summary.append(f"{x-(Hb-H)}, {y+H}, {start_time}, {end_time}, {omega}, {y_stddev}, {omega_stdev}, {sens_ind+1}")

for data in test_summary:
    print(data)

X (cm),Y (cm), start(s), end(s), omega, stdev-Y (cm), stdev-omega, sensor used
38, 168.29754957504315, 50, 65, 3.8279330923324175, 0.2972227771904093, 0.0006454053904566656, 1
38, 161.08741773520455, 97.6, 111, 3.8957376161905737, 0.2972227771904093, 0.00035129141180207795, 1
38, 150.75504615902173, 198.3, 211, 4.052936990990379, 0.2972227771904093, 0.0006897792353052671, 1
38, 131.033826704097, 340.1, 355, 4.408260166970453, 0.2972227771904093, 0.0013360163549427189, 1
38, 122.49875289914758, 421.7, 436, 4.559290808035283, 0.2972227771904093, 0.0011685829787863328, 1
38, 124.12963280525673, 473.5, 485, 4.4638731943291035, 0.2972227771904093, 0.0025331069453084273, 1
38, 117.18734227754771, 502, 515, 4.605222009849808, 0.2972227771904093, 0.0008115647664665646, 1
38, 107.93791127327472, 561.8, 576, 4.814588323461268, 0.2972227771904093, 0.001114927810152933, 1
38, 97.39423212087763, 615.2, 627, 5.164990416573421, 0.2972227771904093, 0.0014368531101292999, 1
38, 91.33928825971498, 686, 

Put Extracted Data in .csv file

In [40]:
# Writing to .csv file
# Write initial data
csv_name = f'hot-water-freq-filtered-{int(cond_diam)}in.csv'
with open(csv_name, 'w') as file_write:
    for test in test_summary:
        file_write.write(test + '\n')